# Master jupyter notebook for LANL - SlimBros Team

Correctly predicting earthquakes is very important for preventing deaths and damage to infrastructure. In this competition we try to predict time left to the next laboratory earthquake based on seismic signal data. Training data represents one huge signal, but in test data we have many separate chunks, for each of which we need to predict time to failure.

## Preliminaries
Let's import everything we need:

In [1]:
import os
import gc
import time
import random
import datetime
import warnings
import feather

import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import lightgbm as lgb
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from catboost import CatBoostRegressor

#from keras import layers, Sequential, callbacks, backend

from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import NuSVR, SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GridSearchCV, cross_val_score
from utils import generate_segment_start_ids, compare_methods
from features import gpi, create_all_features

#Configure the environment
%matplotlib inline
pd.options.display.precision = 15
warnings.filterwarnings('ignore')
random.seed(1013)

Load/compute the necessary features

In [2]:
compute_features = False 
# The computed features are saved in an hdf file along with the time_to_failure to 
# save the time spend reading the training data and the feature computation
#train_data_format = 'csv'
train_data_format = 'feather'

## Training data

In [3]:
def load_train_data(file_format):
    """Load the training dataset."""
    print(f"Loading data from {file_format} file:", end="")
    if file_format.lower() == 'feather':
        train_df = feather.read_dataframe('../input/train.feather')
    else:
        train_df = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16,
                                                            'time_to_failure': np.float32})
        feather.write_dataframe(train_df, '../input/train.feather')
    print("Done")
    return train_df

In [4]:
train = load_train_data(train_data_format)

Loading data from feather file:Done


## Feature generation
- Usual aggregations: mean, std, min and max;
- Average difference between the consequitive values in absolute and percent values;
- Absolute min and max vallues;
- Aforementioned aggregations for first and last 10000 and 50000 values - I think these data should be useful;
- Max value to min value and their differencem also count of values bigger that 500 (arbitrary threshold);
- Quantile features from this kernel: https://www.kaggle.com/andrekos/basic-feature-benchmark-with-quantiles
- Trend features from this kernel: https://www.kaggle.com/jsaguiar/baseline-with-abs-and-trend-features
- Rolling features from this kernel: https://www.kaggle.com/wimwim/rolling-quantiles

In [5]:
saved_files_present = (os.path.isfile('../tmp_results/X_tr.hdf') and 
                       os.path.isfile('../tmp_results/X_test.hdf') and 
                       os.path.isfile('../tmp_results/y_tr.hdf') )

In [6]:
if (not compute_features) and saved_files_present:
    print(f"Reading hdf files:", end="")
    X_tr = pd.read_hdf('../tmp_results/X_tr.hdf', 'data')
    X_test = pd.read_hdf('../tmp_results/X_test.hdf', 'data')
    y_tr = pd.read_hdf('../tmp_results/y_tr.hdf', 'data')  
    print("Done")
else:
    fs = 4000000 #Sampling frequency of the raw signal

    #Compute features for the training data
    segment_size = 150000
    segment_start_ids = generate_segment_start_ids('uniform_no_jump', segment_size, train)
    X_tr = pd.DataFrame(index=range(len(segment_start_ids)), dtype=np.float64)
    y_tr = pd.DataFrame(index=range(len(segment_start_ids)), dtype=np.float64, columns=['time_to_failure'])
    for idx in tqdm_notebook(range(len(segment_start_ids))):        
        seg_id = segment_start_ids[idx]
        seg = train.iloc[seg_id:seg_id + segment_size]
        create_all_features(idx, seg, X_tr, fs)
        y_tr.loc[idx, 'time_to_failure'] = seg['time_to_failure'].values[-1]
    # Sanity check
    means_dict = {}
    for col in X_tr.columns:
        if X_tr[col].isnull().any():
            print(col)
            mean_value = X_tr.loc[X_tr[col] != -np.inf, col].mean()
            X_tr.loc[X_tr[col] == -np.inf, col] = mean_value
            X_tr[col] = X_tr[col].fillna(mean_value)
            means_dict[col] = mean_value

    #Compute features for the test data
    submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
    X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)
    for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
        seg = pd.read_csv('../input/test/' + seg_id + '.csv')
        create_all_features(seg_id, seg, X_test, fs)

    # Sanity check
    for col in X_test.columns:
        if X_test[col].isnull().any():
            X_test.loc[X_test[col] == -np.inf, col] = means_dict[col]
            X_test[col] = X_test[col].fillna(means_dict[col])
            
    X_tr.to_hdf('../tmp_results/X_tr.hdf', 'data')
    X_test.to_hdf('../tmp_results/X_test.hdf', 'data')
    y_tr.to_hdf('../tmp_results/y_tr.hdf', 'data')
    
    del segment_start_ids
    del means_dict
    del submission
    
    print("Done")

Reading hdf files:Done


## Scale data

In [7]:
alldata = pd.concat([X_tr, X_test])
scaler = StandardScaler()
alldata = pd.DataFrame(scaler.fit_transform(alldata), columns=alldata.columns)
X_train_scaled = alldata[:X_tr.shape[0]]
X_test_scaled = alldata[X_tr.shape[0]:]

## Building models

In [8]:
def train_model(X, X_test, y, folds, params=None, model_type='lgb',
                model=None, show_scatter=False):

    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    n_fold = folds.get_n_splits()
    
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        if model_type == 'nn':
            dropout = 0.7
            num_layers = 2
            num_neurons = 128
            model = tf.keras.Sequential()
            model.add(tf.keras.layers.Dense(1024, input_dim=216, activation=tf.nn.relu))
            model.add(tf.keras.layers.Dropout(dropout))
            for l in range(num_layers):
                model.add(tf.keras.layers.Dense(num_neurons, activation=tf.nn.relu))
                model.add(tf.keras.layers.Dropout(dropout))
            model.add(tf.keras.layers.Dense(1))
            model = tf.keras.utils.multi_gpu_model(model, gpus=2, cpu_merge=False)
            model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
            EPOCHS = 1000
            early_stop = tf.keras.callbacks.EarlyStopping(monitor='mean_absolute_error', patience=100)

            history = model.fit(
                X_train,
                y_train,
                epochs=EPOCHS,
                validation_data = (X_valid, y_valid), 
                verbose=0,
                callbacks=[early_stop, PrintDot()])
            hist = pd.DataFrame(history.history)
            val_score = hist['val_mean_absolute_error'].iloc[-1]
            print(f'val_score={val_score}')
            plot_history(history)
        
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            y_pred = model.predict(X_test).reshape(-1,)

            score = mean_absolute_error(y_valid, y_pred_valid)
            print(f'Fold {fold_n}. MAE: {score:.4f}.')
        
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = 50000, n_jobs = 32)
            model.fit(X_train, y_train, 
                      eval_set=[(X_train, y_train), (X_valid, y_valid)],
                      eval_metric='mae',
                      verbose=10000,
                      early_stopping_rounds=2000)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data,
                              num_boost_round=20000,
                              evals=watchlist,
                              early_stopping_rounds=200,
                              verbose_eval=500,
                              params=params)

            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns),
                                         ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns),
                                   ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = mean_absolute_error(y_valid, y_pred_valid)
            y_pred = model.predict(X_test).reshape(-1,)
            print(f'Fold {fold_n}. MAE: {score:.4f}.')
            print('')
        
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric='MAE', task_type='GPU', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True,
                      verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        if model_type == 'gdi':
            y_pred_valid = gpi(X_valid).values
            y_pred = gpi(X_test).values
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_absolute_error(y_valid, y_pred_valid))
        
        prediction += y_pred    
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance['feature'] = X.columns
            fold_importance['importance'] = model.feature_importances_
            fold_importance['fold'] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    if show_scatter:
        fig, axis = plt.subplots(1, 2, figsize=(12,5))
        ax1, ax2 = axis
        ax1.set_xlabel('actual')
        ax1.set_ylabel('predicted')
        ax2.set_xlabel('train index')
        ax2.set_ylabel('time to failure')
        
        ax1.scatter(y, oof, color='brown')
        ax1.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)], color='blue')

        ax2.plot(y, color='blue', label='y_train')
        ax2.plot(oof, color='orange')
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance['importance'] /= n_fold
        return oof, prediction, np.mean(scores), np.std(scores), feature_importance
    else:
        return oof, prediction, np.mean(scores), np.std(scores)


In [9]:
n_fold = 5
folds_models = KFold(n_splits=n_fold, shuffle=True, random_state=11)

Let's try a few different models and submit the one with the best validation score. The predicted values in the following plots are using a out-of-fold scheme.

### LGBM (Gradient Boosting)
Gradient boosting that uses tree based learning algorithms.

In [ ]:
params_lgb = {
    'objective': 'huber',
    'boosting': 'gbdt',
    'verbosity': -1,
    'num_leaves': 8, #54,
    'min_data_in_leaf': 100, #79,
    'max_depth': 6, #-1,
    'learning_rate': 0.01,
    'bagging_freq': 3, #5,
    'bagging_fraction': 0.7166666666666667, #0.8126672064208567,
    'bagging_seed': 11,
    'metric': 'mae',
    'reg_alpha': 0.19444444444444445, #0.1302650970728192,
    'reg_lambda': 0.8555555555555555 #0.3603427518866501
}
oof_lgb, prediction_lgb, score_mean_lgb, score_std_lgb, feature_importance = train_model(X=X_train_scaled,
                                                                                         X_test=X_test_scaled,
                                                                                         y=y_tr,
                                                                                         folds=folds_models,
                                                                                         params=params_lgb,
                                                                                         model_type='lgb',
                                                                                         show_scatter=True)

### XGBoost (Gradient Boosting)
Gradient boosting that uses tree based learning algorithms.

In [ ]:
params_xgb = {
    'eta': 0.01, #Andrew uses 0.03
    'max_depth': 6, #Andrew uses 10
    'subsample': 0.5, #Andrew uses 0.9
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'silent': True,
    'nthread': 32
} # CV mean score: 2.0801, std: 0.0711.
oof_xgb, prediction_xgb, score_mean_xgb, score_std_xgb = train_model(X=X_train_scaled,
                                                                     X_test=X_test_scaled,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     params=params_xgb,
                                                                     model_type='xgb',
                                                                     show_scatter=True)

### SVM
Similar to NuSVC, for regression, uses a parameter nu to control the number of support vectors. However, unlike NuSVC, where nu replaces C, here nu replaces the parameter epsilon of epsilon-SVR.

In [ ]:
#model = NuSVR(gamma='scale', nu=0.9, C=10.0, tol=0.01) #original values
model = NuSVR(gamma='scale', nu=0.63, C=0.4469387755102041, tol=0.01)
oof_svr, prediction_svr, score_mean_svr, score_std_svr = train_model(X=X_train_scaled,
                                                                     X_test=X_test_scaled,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     params=None,
                                                                     model_type='sklearn',
                                                                     model=model,
                                                                     show_scatter=True)
del model

### Catboost

In [ ]:
params_cat = {
    'loss_function':'MAE'
}
oof_cat, prediction_cat, score_mean_cat, score_std_cat = train_model(X=X_train_scaled,
                                                                     X_test=X_test_scaled,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     params=params_cat,
                                                                     model_type='cat',
                                                                     show_scatter=True)

### Kernel Ridge
This model combines regularized linear regression with a given kernel (radial basis in this case).

In [ ]:
#model = KernelRidge(kernel='rbf', alpha=0.1, gamma=0.01) #Original parameters
model = KernelRidge(kernel='rbf', alpha=2.4497346938775513, gamma=0.0018461224489795917)
oof_r, prediction_r, score_mean_r, score_std_r = train_model(X=X_train_scaled,
                                                             X_test=X_test_scaled,
                                                             y=y_tr,
                                                             folds=folds_models,
                                                             params=None,
                                                             model_type='sklearn',
                                                             model=model,
                                                             show_scatter=True)
del model

### Genetic Program Model
Genetic programming model from https://www.kaggle.com/scirpus/andrews-script-plus-a-genetic-program-model

In [ ]:
# There are no tuning here. We took at as face value.
oof_gdi, prediction_gdi, score_mean_gdi, score_std_gdi = train_model(X=X_train_scaled,
                                                                     X_test=X_test_scaled,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     model_type='gdi',
                                                                     show_scatter=True)

### Random Forest
This regressor fits many decision trees with different subsets of the original data and average the predictions between them.

In [ ]:
params_rf = {
    'max_depth': 12, #8,
    'max_features': 'log2', #'auto',
    'min_samples_leaf': 2,
    'min_samples_split': 4 #6
} #CV mean score: 2.0448, std: 0.0793.
model = RandomForestRegressor(criterion='mae', n_estimators=200, n_jobs=-1, **params_rf)
oof_rf, prediction_rf, score_mean_rf, score_std_rf = train_model(X=X_train_scaled,
                                                                 X_test=X_test_scaled,
                                                                 y=y_tr,
                                                                 folds=folds_models,
                                                                 params=params_rf,
                                                                 model_type='sklearn',
                                                                 model=model,
                                                                 show_scatter=True)
del model

### Extremely Randomized Trees


In [ ]:
params_ex = {
    'max_depth': 12,
    'max_features': 'sqrt',
    'min_samples_leaf': 2,
    'min_samples_split': 4
}
model = ExtraTreesRegressor(criterion='mae', n_estimators=200, n_jobs=-1, **params_ex)
oof_ex, prediction_ex, score_mean_ex, score_std_ex = train_model(X=X_train_scaled,
                                                                 X_test=X_test_scaled,
                                                                 y=y_tr,
                                                                 folds=folds_models,
                                                                 params=params_ex,
                                                                 model_type='sklearn',
                                                                 model=model,
                                                                 show_scatter=True)
del model

### Ada Boost
AdaBoost begins by fitting a base estimator on the original dataset and then fits additional copies on the same dataset. At each iteration (estimator), the weights of instances are adjusted according to the error of the last prediction. It's similar to the next model, but gradient boosting fits additional estimator copies on the current error and not on the original dataset.

In [ ]:
params_ada = {
    'learning_rate': 0.001
}
base = KernelRidge(kernel='rbf', alpha=2.4497346938775513, gamma=0.0018461224489795917)
model = AdaBoostRegressor(base_estimator=base, n_estimators=100, **params_ada)
oof_ada, prediction_ada, score_mean_ada, score_std_ada = train_model(X=X_train_scaled,
                                                                     X_test=X_test_scaled,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     params=params_ada,
                                                                     model_type='sklearn',
                                                                     model=model,
                                                                     show_scatter=True)
del base
del model

### NeuralNet
This is a simple FC model mimiquing the previous genetic algorithm.

In [10]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print('')
        else:
            print('.', end='')

In [11]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
  
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
             label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
             label = 'Val Error')
    plt.ylim([0,5])
    plt.legend()
    plt.grid()
    plt.show()

In [13]:
dropout = 0.7
num_layers = 2
num_neurons = 128
#with tf.device('/cpu:0'):
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024, input_dim=216, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(dropout))
for l in range(num_layers):
    model.add(tf.keras.layers.Dense(num_neurons, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dropout(dropout))
model.add(tf.keras.layers.Dense(1))
#model = tf.keras.utils.multi_gpu_model(model, gpus=1, cpu_relocation=True)
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
EPOCHS = 1000
early_stop = tf.keras.callbacks.EarlyStopping(monitor='mean_absolute_error', patience=100)
history = model.fit(
    X_train_scaled,
    y_tr,
    validation_split=0.5,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[early_stop, PrintDot()])
hist = pd.DataFrame(history.history)
val_score = hist['val_mean_absolute_error'].iloc[-1]
print(f'val_score={val_score}')
plot_history(history)

Train on 2089 samples, validate on 2089 samples
Epoch 1/1000
2089/2089 [==============================] - 1s 341us/sample - loss: 3.8408 - mean_absolute_error: 3.8408 - val_loss: 2.3732 - val_mean_absolute_error: 2.3732
Epoch 2/1000
2089/2089 [==============================] - 0s 155us/sample - loss: 3.3488 - mean_absolute_error: 3.3488 - val_loss: 2.2817 - val_mean_absolute_error: 2.2817
Epoch 3/1000
2089/2089 [==============================] - 0s 156us/sample - loss: 3.2710 - mean_absolute_error: 3.2710 - val_loss: 2.6445 - val_mean_absolute_error: 2.6445
Epoch 4/1000
2089/2089 [==============================] - 0s 150us/sample - loss: 3.1756 - mean_absolute_error: 3.1756 - val_loss: 2.7095 - val_mean_absolute_error: 2.7095
Epoch 5/1000
2089/2089 [==============================] - 0s 155us/sample - loss: 3.0286 - mean_absolute_error: 3.0286 - val_loss: 2.3684 - val_mean_absolute_error: 2.3684
Epoch 6/1000
2089/2089 [==============================] - 0s 154us/sample - loss: 2.9917 - m

KeyboardInterrupt: 

In [ ]:
oof_nn, prediction_nn, score_mean_nn, score_std_nn = train_model(X=X_train_scaled,
                                                                 X_test=X_test_scaled,
                                                                 y=y_tr,
                                                                 folds=folds_models,
                                                                 model_type='nn',
                                                                 show_scatter=True)

### CNN
This is a simple CNN model.

## Stacking and blending
And now let's try stacking :) We can use the same function for it.

In [ ]:
train_stack = np.vstack([oof_lgb, oof_xgb, oof_svr, oof_r,
                         oof_cat, oof_gdi, oof_rf, oof_ex,
                         oof_ada]).transpose()
train_stack = pd.DataFrame(train_stack, columns = ['lgb', 'xgb', 'svr', 'r', 'cat', 'gdi', 'rf', 'ex', 'ada'])
test_stack = np.vstack([prediction_lgb, prediction_xgb, prediction_svr, prediction_r,
                        prediction_cat, prediction_gdi, prediction_rf, prediction_ex,
                        prediction_ada]).transpose()
test_stack = pd.DataFrame(test_stack)

In [ ]:
params_lgb_stack = {
    'objective': 'huber',
    'boosting': 'gbdt',
    'verbosity': -1,
    'num_leaves': 8, #54,
    'min_data_in_leaf': 100, #79,
    'max_depth': 5, #-1,
    'learning_rate': 0.01, #0.01,
    'bagging_freq': 5, #5,
    'bagging_fraction': 0.7555555555555555, #0.8126672064208567,
    'bagging_seed': 11,
    'metric': 'mae',
    'reg_alpha': 0.95, #0.1302650970728192,
    'reg_lambda': 0.19444444444444445 #0.3603427518866501
}
oof_lgb_stack, prediction_lgb_stack, score_mean_lgb_stack, score_std_lgb_stack, feature_importance_lgb_stack = train_model(
    X=train_stack,
    X_test=test_stack,
    y=y_tr,
    folds=folds_models,
    params=params_lgb_stack,
    model_type='lgb')

In [ ]:
cols = feature_importance_lgb_stack[['feature', 'importance']].groupby('feature').mean().sort_values(
    by='importance', ascending=False).index
best_features = feature_importance_lgb_stack.loc[feature_importance_lgb_stack.feature.isin(cols)]
plt.figure(figsize=(16, 8));
sns.barplot(x='importance', y='feature', data=best_features.sort_values(by='importance', ascending=False));
plt.title('LGB Features (avg over folds)');

In [ ]:
oof_blend = (oof_lgb + oof_xgb + oof_svr + oof_r + oof_cat + oof_gdi + oof_rf + oof_ex + oof_ada) / 9
prediction_blend = (prediction_lgb + prediction_xgb + prediction_svr 
                    + prediction_cat + prediction_r + prediction_gdi
                    + prediction_rf + prediction_ex + prediction_ada) / 9
score_mean_blend = (score_mean_lgb + score_mean_xgb + score_mean_svr + score_mean_cat
                    + score_mean_r + score_mean_gdi + score_mean_rf + score_mean_ex + score_mean_ada) / 9
score_std_blend = (score_std_lgb + score_std_xgb + score_std_svr + score_std_cat
                    + score_std_r + score_std_gdi + score_std_rf + score_std_ex + score_std_ada) / 9

### Submissions

In [ ]:
evaluation = pd.DataFrame(dtype=np.float64)
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
my_methods = ['lgb', 'xgb', 'svr', 'r', 'cat', 'gdi', 'rf', 'ex', 'ada', 'lgb_stack', 'blend']

In [ ]:
for method in my_methods:
    submission['time_to_failure'] = globals()['prediction_' + method]
    submission.to_csv('../output/submission_' + method + '.csv')
    evaluation.loc[method, 'mean'] = globals()['score_mean_' + method]
    evaluation.loc[method, 'std'] = globals()['score_std_' + method]
evaluation.to_csv(f"../output/evaluation_{str(datetime.datetime.now())}.csv")

In [ ]:
compare_methods(evaluation)